In [8]:
import os
import shutil
import random
import pandas as pd

In [9]:
Clasif = pd.read_csv('Carac_Stars.csv')
Clasif

,Kepler_ID,Metalicidad,Clasificacion,incert_clas
0,10797460,0.136,CANDIDATE,1.000
1,10811496,-0.127,FALSE POSITIVE,0.000
2,10848459,-0.522,FALSE POSITIVE,0.000
3,10854555,0.081,CANDIDATE,1.000
4,10872983,-0.067,CANDIDATE,0.992
...,...,...,...,...
7677,10128825,-0.046,CANDIDATE,0.497
7678,10147276,-0.038,FALSE POSITIVE,0.021
7679,10156110,0.082,FALSE POSITIVE,0.000
7680,8460634,-0.556,FALSE POSITIVE,0.000


In [33]:
Clasif[Clasif['Kepler_ID']==10000162]['Clasificacion']

'3845    CANDIDATE\nName: Clasificacion, dtype: object'

In [43]:
def mover_archivos(archivos:list, vieja_carpeta:str, nueva_carpeta:str):
    """Recibe una lista de archivos, la carpeta actual y el lugar a donde se moveran"""
    for archivo in archivos:
        shutil.move(vieja_carpeta+'/'+archivo, nueva_carpeta+'/'+archivo)

def segmentacion_estrellas(base_dir:str, df_clasif, columna_clasif:str, columna_ID:str):
    """Se separan los datos en dos carpetas ubicadas en la carpeta orginal, 
    una carpeta con los candidatos a tener exoplanetas y otra a los que no 
    poseen"""
    # Listar todos los archivos en la carpeta base
    archivos = [f for f in os.listdir(base_dir) if f.endswith('.csv') and os.path.isfile(os.path.join(base_dir, f))]
    archivos_1 = []
    archivos_0 = []
    for archivo in archivos:
        print(archivo)
        print(archivo[3:-4])
        if df_clasif[df_clasif[columna_ID]==int(archivo[3:-4])][columna_clasif].iloc[0]=='CANDIDATE':
            archivos_1.append(archivo)
        else:
            archivos_0.append(archivo)
    mover_archivos(archivos_1, base_dir, base_dir+'/1')
    mover_archivos(archivos_0, base_dir, base_dir+'/0')
    
def modeling_div(base_dir:str, proporciones, new_carpeta:str):
    # Listar todos los archivos en la carpeta base
    archivos = [f for f in os.listdir(base_dir)]
    random.shuffle(archivos)  # Mezclar los archivos aleatoriamente
    
    # Calcular cantidades de archivos para cada subcarpeta
    total_archivos = len(archivos)
    num_train = int(total_archivos * proporciones['train'])
    num_val = int(total_archivos * proporciones['val'])
    
    # Dividir los archivos en las subcarpetas correspondientes
    archivos_train = archivos[:num_train]
    archivos_val = archivos[num_train:num_train + num_val]
    archivos_test = archivos[num_train + num_val:]
    
    mover_archivos(archivos_train, base_dir, new_carpeta['train'])
    mover_archivos(archivos_val, base_dir, new_carpeta['val'])
    mover_archivos(archivos_test, base_dir, new_carpeta['test'])

In [46]:
old_dir = '../Curvas_de_luz_God'

proporciones = {'train': 0.7, 
                'val': 0.2, 
                'test': 0.1}

new_carpeta_0 = {'train': '../Datos_Exoplanetas/0/train', 
               'val':'../Datos_Exoplanetas/0/Validation', 
               'test':'../Datos_Exoplanetas/0/test'}

new_carpeta_1 = {'train': '../Datos_Exoplanetas/1/train', 
               'val':'../Datos_Exoplanetas/1/Validation', 
               'test':'../Datos_Exoplanetas/1/test'}

In [ ]:
segmentacion_estrellas(old_dir, Clasif, 'Clasificacion', 'Kepler_ID')

In [47]:
modeling_div(old_dir+'/1', proporciones, new_carpeta_1)

In [48]:
modeling_div(old_dir+'/0', proporciones, new_carpeta_0)